In [5]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
path = '/media/onno/Algemeen/Thesis/GFS_T850/'

In [8]:
df_MAE_GFS = pd.read_csv(path+'MAE/Mean_Absolute_Error_lead_day_5_lon_6_14_lat_54_46_GFS.csv',index_col=0)
df_MAE_GFS.index = pd.to_datetime(df_MAE_GFS.index)
df_MAE_ERA5RF = pd.read_csv(path+'MAE/Mean_Absolute_Error_lead_day_5_lon_6_14_lat_54_46_ERA5RF.csv',index_col=0)
df_MAE_ERA5RF.index = pd.to_datetime(df_MAE_ERA5RF.index)
df_GSS_GFS = pd.read_csv(path+'GSS/Gilbert_Skill_Score_lead_day_5_lon_6_14_lat_54_46_GFS.csv',index_col=0)
df_GSS_GFS.index = pd.to_datetime(df_GSS_GFS.index)
df_GSS_ERA5RF = pd.read_csv(path+'GSS/Gilbert_Skill_Score_lead_day_5_lon_6_14_lat_54_46_ERA5RF.csv',index_col=0)
df_GSS_ERA5RF.index = pd.to_datetime(df_GSS_ERA5RF.index)





In [9]:
columnz = ['persistent_hw','short_hw','persistent_cw','short_cw']
total_GFS = []; total_ERA5RF = []
for column in columnz:
    column_MAE = column+'_MAE'
    df_MAE = df_MAE_GFS.loc[:,[column]].dropna().rename(columns={column:column_MAE})
    df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
    df_MAE = df_MAE.sort_values(['rank'])
    column_GSS = column+'_GSS'
    df_GSS = df_GSS_GFS.loc[:,[column]].dropna().rename(columns={column:column_GSS})
    df_GSS['rank'] = df_GSS[column_GSS].rank(method='min',ascending=False)
    df_GSS = df_GSS.sort_values(['rank'])

    df_combined_rank = df_MAE.copy()
    df_combined_rank[column_GSS] = df_GSS[column_GSS]
    df_combined_rank=df_combined_rank.reindex(columns=[column_MAE,column_GSS,'rank'])
    df_combined_rank['rank']=((df_GSS['rank']+df_MAE['rank'])/2)
    df_combined_rank = df_combined_rank.sort_values(['rank'])
    df_combined_rank.to_csv(path+'rank_forecast/forecast_ranking_lead_day_5_{}_GFS.csv'.format(column))
    total_GFS.append(df_combined_rank)
    
    column_MAE = column+'_MAE'
    df_MAE = df_MAE_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_MAE})
    df_MAE['rank'] = df_MAE[column_MAE].rank(method='min')
    df_MAE = df_MAE.sort_values(['rank'])
    column_GSS = column+'_GSS'
    df_GSS = df_GSS_ERA5RF.loc[:,[column]].dropna().rename(columns={column:column_GSS})
    df_GSS['rank'] = df_GSS[column_GSS].rank(method='min',ascending=False)
    df_GSS = df_GSS.sort_values(['rank'])

    df_combined_rank = df_MAE.copy()
    df_combined_rank[column_GSS] = df_GSS[column_GSS]
    df_combined_rank=df_combined_rank.reindex(columns=[column_MAE,column_GSS,'rank'])
    df_combined_rank['rank']=((df_GSS['rank']+df_MAE['rank'])/2)
    df_combined_rank = df_combined_rank.sort_values(['rank'])
    df_combined_rank.to_csv(path+'rank_forecast/forecast_ranking_lead_day_5_{}_ERA5RF.csv'.format(column))
    total_ERA5RF.append(df_combined_rank)

    
